<a href="https://colab.research.google.com/github/mgaiduk/mgaiduk/blob/main/grasp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install  "tensorflow-text==2.13.*"
%pip install "tf-models-official==2.13.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 25.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=dff1bd98df25134c85363ed75b93c018d85e49823f6931a9dd8633c166a90bf3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.metrics import f1_score

import json
from sklearn.metrics import f1_score

tf.get_logger().setLevel('ERROR')
print(tf.__version__)
tf.config.list_physical_devices()

2.13.0


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
prompts = []
labels = []
with open("/content/drive/MyDrive/Colab Notebooks/samples-1680.jsonl", "r") as file:
  for line in file:
    js = json.loads(line)
    if not "HR" in js:
      continue
    prompts.append(js["prompt"])
    labels.append(js["HR"])

In [6]:
from sklearn.model_selection import train_test_split
train_prompts, test_prompts, train_labels , test_labels = train_test_split(prompts, labels, test_size = 0.2)

In [7]:
train_prompts = tf.constant(train_prompts)
train_labels = tf.constant(train_labels)
test_prompts = tf.constant(test_prompts)
test_labels = tf.constant(test_labels)
train_ds = tf.data.Dataset.from_tensor_slices((train_prompts, train_labels)).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((test_prompts, test_labels)).batch(32)
for r in train_ds:
  break
r

(<tf.Tensor: shape=(32,), dtype=string, numpy=
 array([b'In Great Britain, for example, some most pertinent thoughts on this subject were expressed in a notable Reservation to the Majority Report of the <Organization>, in the following language: "On a review of the evidence we cannot but feel that there is considerable confusion in certain quarters with regard to the relation which exists between the care of the individual\'s health and the wider question of the promotion of the health of the community.',
        b'Objective:\n\nTo evaluate the performance of real-time CAD with endocytoscopes (\xc3\x97520 ultramagnifying colonoscopes providing microvascular and cellular visualization of colorectal polyps after application of the narrow-band imaging [NBI] and methylene blue staining modes, respectively).',
        b'Without the embers burn a few times complete my landlady breakfasts were beau. I had said i noticed with that that taut jeans and were humored by and secure, is too. We stop

In [8]:
#bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
bert_model_name = "bert_en_uncased_L-12_H-768_A-12"

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [9]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [10]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.92169887 -0.39353466 -0.53931785  0.6825625   0.43848515 -0.14021172
  0.87747127  0.26043358 -0.6311301  -0.9999658  -0.26320052  0.85105324]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[ 0.19451568  0.25141704  0.19075024 ... -0.24845096  0.38568568
   0.13291004]
 [-0.5947861  -0.3942036   0.25245717 ... -0.7694675   1.1564167
   0.32475695]
 [ 0.00641518 -0.15766475  0.5461027  ... -0.17451018  0.6028964
   0.42672247]
 ...
 [ 0.21948306 -0.20927139  0.5386825  ...  0.24693541  0.1825099
  -0.44427088]
 [ 0.01080238 -0.44553143  0.35990992 ...  0.3172276   0.2356278
  -0.630706  ]
 [ 0.2932116  -0.10581895  0.6114753  ...  0.2074579   0.14494637
  -0.35353395]]


In [11]:
preprocessor = hub.load(tfhub_handle_preprocess)
def build_classifier_model():
  input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='input')
  tokenize = hub.KerasLayer(preprocessor.tokenize)
  tokenized_inputs = tokenize(input)
  seq_length = 128  # Your choice here.
  bert_pack_inputs = hub.KerasLayer(
    preprocessor.bert_pack_inputs,
    arguments=dict(seq_length=seq_length))  # Optional argument.
  encoder_inputs = bert_pack_inputs([tokenized_inputs])
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='dense1')(net)
  return tf.keras.Model(input, net)
classifier_model = build_classifier_model()
classifier_model(r[0])

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[-0.04249637],
       [-0.82887995],
       [ 0.22077744],
       [-0.68148506],
       [-0.25129414],
       [-0.4411524 ],
       [-0.03555377],
       [-0.6815258 ],
       [-0.1692827 ],
       [-0.44889873],
       [-0.12560216],
       [ 0.04775489],
       [-0.73498803],
       [-0.870476  ],
       [-0.32503855],
       [-0.41160017],
       [ 0.26620483],
       [-0.6070614 ],
       [ 0.17717999],
       [-0.752297  ],
       [ 0.05057841],
       [-0.59428227],
       [-0.38527995],
       [-0.8160822 ],
       [-0.6270472 ],
       [-0.4832495 ],
       [-0.7552974 ],
       [-0.51741123],
       [-0.4210317 ],
       [-0.7238193 ],
       [-0.6160564 ],
       [-0.613596  ]], dtype=float32)>

In [12]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [13]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [14]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
Epoch 1/5
37/37 [==============================] - 53s 955ms/step - loss: 0.3104 - binary_accuracy: 0.9472 - val_loss: 0.2188 - val_binary_accuracy: 0.9377
Epoch 2/5
37/37 [==============================] - 38s 1s/step - loss: 0.1689 - binary_accuracy: 0.9498 - val_loss: 0.1670 - val_binary_accuracy: 0.9377
Epoch 3/5
37/37 [==============================] - 40s 1s/step - loss: 0.1016 - binary_accuracy: 0.9506 - val_loss: 0.1457 - val_binary_accuracy: 0.9377
Epoch 4/5
37/37 [==============================] - 37s 1s/step - loss: 0.0405 - binary_accuracy: 0.9775 - val_loss: 0.1540 - val_binary_accuracy: 0.9619
Epoch 5/5
37/37 [==============================] - 37s 1s/step - loss: 0.0256 - binary_accuracy: 0.9887 - val_loss: 0.1593 - val_binary_accuracy: 0.9654


In [15]:
classifier_model.save("/content/drive/MyDrive/Colab Notebooks/data/hr_moderation_model.wad")

In [17]:
# calc f1 score
all_preds = []
all_labels = []
for batch in val_ds:
  preds = classifier_model(batch[0])
  for pred in preds:
    p = pred[0].numpy()
    all_preds.append(pred)
  for label in batch[1]:
    all_labels.append(label)
len(all_preds), len(all_labels), all_preds[:5], all_labels[:5]

(289,
 289,
 [<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-4.8126717], dtype=float32)>,
  <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-8.292546], dtype=float32)>,
  <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-5.298879], dtype=float32)>,
  <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-6.739436], dtype=float32)>,
  <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-8.063936], dtype=float32)>],
 [<tf.Tensor: shape=(), dtype=int32, numpy=0>,
  <tf.Tensor: shape=(), dtype=int32, numpy=0>,
  <tf.Tensor: shape=(), dtype=int32, numpy=0>,
  <tf.Tensor: shape=(), dtype=int32, numpy=0>,
  <tf.Tensor: shape=(), dtype=int32, numpy=0>])

In [18]:
# finding out the best threshold for f1 score
threshold = 0.0
while threshold <= 1.0:
  def thr(value, threshold):
    if value >= threshold:
      return 1
    return 0

  predicted_labels = [thr(value, threshold) for value in all_preds]
  f1 = f1_score(all_labels, predicted_labels)
  print("Threshold: ", threshold, "; f1: ", f1)
  threshold += 0.05

Threshold:  0.0 ; f1:  0.6875000000000001
Threshold:  0.05 ; f1:  0.6875000000000001
Threshold:  0.1 ; f1:  0.6875000000000001
Threshold:  0.15000000000000002 ; f1:  0.6875000000000001
Threshold:  0.2 ; f1:  0.6875000000000001
Threshold:  0.25 ; f1:  0.6875000000000001
Threshold:  0.3 ; f1:  0.6875000000000001
Threshold:  0.35 ; f1:  0.6875000000000001
Threshold:  0.39999999999999997 ; f1:  0.6875000000000001
Threshold:  0.44999999999999996 ; f1:  0.6875000000000001
Threshold:  0.49999999999999994 ; f1:  0.6875000000000001
Threshold:  0.5499999999999999 ; f1:  0.6875000000000001
Threshold:  0.6 ; f1:  0.6875000000000001
Threshold:  0.65 ; f1:  0.6451612903225806
Threshold:  0.7000000000000001 ; f1:  0.6451612903225806
Threshold:  0.7500000000000001 ; f1:  0.6451612903225806
Threshold:  0.8000000000000002 ; f1:  0.6451612903225806
Threshold:  0.8500000000000002 ; f1:  0.6451612903225806
Threshold:  0.9000000000000002 ; f1:  0.6451612903225806
Threshold:  0.9500000000000003 ; f1:  0.6451